### 크롤링 코드
현대자동차의 차종별 오너평가의 리뷰를 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request as req
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

User_Agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
chrome_options = Options()
chrome_options.add_argument(f"user-agent={User_Agent }")
driver = webdriver.Chrome()


car_list = list()
                          
for i in car['차종'][:3]:
    driver.get('https://www.naver.com/')
    driver.maximize_window()
    
    keyword = i + ' 오너평가'

    search = driver.find_element(By.CSS_SELECTOR,'#query')
    search.send_keys(keyword)
    search.send_keys('\n')
    time.sleep(5)
    try:
        # 스크롤을 내리고자 하는 div 요소를 찾기 (XPath는 변경해야 함)
        scrollable_div = driver.find_element(By.XPATH, "/html/body/div[3]/div[2]/div/div[1]/div[2]/div[2]/div/div/div[5]")

        # 이전 스크롤 위치 초기화
        last_scroll_position = -1

        while True:
            # 현재 스크롤 위치 확인
            current_scroll_position = driver.execute_script("return arguments[0].scrollTop", scrollable_div)

            # 스크롤을 끝까지 내렸는지 확인
            if current_scroll_position == last_scroll_position:
                break

            # 스크롤을 끝까지 내림
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)

            # 현재 스크롤 위치를 이전 스크롤 위치로 업데이트
            last_scroll_position = current_scroll_position

            # 스크롤이 완료될 때까지 대기
            time.sleep(1)  # 적절한 대기 시간을 설정하세요
        time.sleep(10)
        person = list()
        reviews = driver.find_elements(By.CSS_SELECTOR,'#cbox_module_wai_u_cbox_content_wrap_tabpanel > ul > li')
        for review in reviews:
            person.append(review.text)
        
        car_list.append(person)
    except:
        car_list.append([])
driver.quit()

### 트위터 감정 데이터 전처리
Kaggle의 tweet_emotion 데이터를 이용해 감정 모델을 구축 <br/>
-> RoBERTa모델로 ①content + ② sentiment의 관계를 학습시키기 위해 텍스트 전처리와 라벨링을 진행

In [1]:
# 텍스트 전처리 코드

def clean_text(val):
    val = misspelled_correction(val)
    val = cont_to_meaning(val)
    val = p.clean(val)
    val = ' '.join(punctuation(emoji.demojize(val)).split())
    return val

### 리뷰 데이터 전처리
트위터 데이터 전처리를 위해 사용했던 함수를 이용해 전처리 수행<br/>
현재 리뷰 데이터는 한글로 작성되어 있기 때문에 트위터 데이터를 활용한 모델에 적용시키기 위해서는 영어로 번역하는 과정을 거쳐야 함<br/>
Google Translate API를 이용해 번역 진행

In [2]:
def review_to_en(text,target_language='en'):
    print(text)

    # Fake User Agent 생성
    user_agent = UserAgent()
    headers = {'User-Agent': user_agent.random}

    # Google Translate URL 및 매개변수 설정
    translate_url = 'https://translate.googleapis.com/translate_a/single'
    params = {
        'client': 'gtx',
        'sl': 'auto',
        'tl': target_language,
        'dt': 't',
        'q': text,
    }

    # HTTP 요청 보내기
    response = requests.get(translate_url, params=params, headers=headers)

    # 결과 출력
    if response.status_code == 200:
        result = response.json()
        translated_text = result[0][0][0]
        print(f'Translated Text ({target_language}): {translated_text}')
    else:
        print(f'Error: {response.status_code}')

    return translated_text

### 모델링
RoBERTa모델을 이용해 학습 진행

1) 모델 정의

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer
from transformers import TFAutoModel

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        #padding=True,
        max_length=maxlen
    )

    return np.array(enc_di['input_ids'])

def build_model(transformer, max_len=160):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(3, activation='softmax')(cls_token)

    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

AUTO = tf.data.experimental.AUTOTUNE
MODEL = 'roberta-base'

tokenizer = AutoTokenizer.from_pretrained(MODEL) # MODEL에서의 토크나이저를 정의

X_train_t = regular_encode(X_train.values.tolist(), tokenizer, maxlen=max_len)
X_test_t = regular_encode(X_test.values.tolist(), tokenizer, maxlen=max_len)

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train_t, y_train))
    .repeat()
    .shuffle(1995)
    .batch(batch_size)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test_t, y_test))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)


transformer_layer = TFAutoModel.from_pretrained(MODEL)

model_roberta_base = build_model(transformer_layer, max_len=max_len)
model_roberta_base.summary()

2) 모델학습<br/>
트위터 데이터로 학습된 모델을 사용하여 review에서 positive, neutral, negative가 나올 확률 도출

In [ ]:
n_steps = X_train.shape[0] // batch_size
with tf.device('/gpu:0'):
  model_roberta_base.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=Epoch)

3) sentiment 확률값 도출

In [ ]:
def get_sentiment2(model,text):
    text = clean_text(text)
    #tokenize
    x_test1 = regular_encode([text], tokenizer, maxlen=max_len)
    test1 = (tf.data.Dataset.from_tensor_slices(x_test1).batch(1))
    #test1
    sentiment = model.predict(test1,verbose = 0)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

ex)
result =get_sentiment2(model_roberta_base,"I hate this game so much,It make me angry all the time ")
# positive -> 19.0 / negative-> 81.0

4) 리뷰데이터에 합치기

In [ ]:
for num,i in enumerate(review['en']):
  result =get_sentiment2(model_roberta_base,i)
  review['result_dict_robert'][num] = dict(zip(result['sentiment'],result['percentage']))